# **RNSA Breast Cancer Detector.**

## ML:

**Librerias**

In [22]:
#Librerias

import pandas as pd
import tensorflow as tf
import pydicom
from pathlib import Path

#Constantes
IMGFOLDER_PATH = "../data/raw/train_images"


In [34]:
def ReadData():
    print("Reading Dataset...\n==========\n")

    try:
        data = pd.read_csv("../data/processed/clean_data.csv")
        print(f"{data.head(5)}\n\n · Successfully data readed")
        return data
    except Exception as e:
        print(f"...Failed data readed. Error: {e}")
        return None

data = ReadData()

Reading Dataset...

   patient_id    image_id   age  cancer  biopsy  invasive  BIRADS  implant  \
0       10006   462822612  61.0       0       0         0     4.0        0   
1       10006  1459541791  61.0       0       0         0     4.0        0   
2       10006  1864590858  61.0       0       0         0     4.0        0   
3       10006  1874946579  61.0       0       0         0     4.0        0   
4       10011   220375232  55.0       0       0         0     0.0        0   

   density  laterality  view  
0      2.0         0.0   0.0  
1      2.0         0.0   1.0  
2      2.0         1.0   1.0  
3      2.0         1.0   0.0  
4      1.0         0.0   0.0  

 · Successfully data readed


In [57]:
def GetPaths(root, dataset):

    #Creamos una lista con el Path de las imagenes.

    root = Path(root) # conversion Str -> Path
    list_path = list(root.rglob("*.dcm")) # lista de paths

    #Mapeamos
    map_label = {}

    for p in list_path:
        patient_id = p.parts[-2] # -> folder del paciente donde están las imagenes.
        img_id = p.stem # -> la imagen en cuestion.

        try:    
            filter = dataset[(dataset["patient_id"].astype(str) == patient_id) & (dataset["image_id"].astype(str) == img_id)] #Importante convertir los valores del dataset en str para la comparacion.

            if not filter.empty:
                target = int(filter["cancer"].values[0]) # -> tomamos el valor del target.
                map_label[p] = target
            else:
                print("No coincidences in dataset.")
        except Exception as e:
            print(f"Error: {e} in row {p}.")

    return map_label



mappingIMG = GetPaths(IMGFOLDER_PATH, data)